<a href="https://colab.research.google.com/github/manmohangithub/FMML_PROJECT_AND_LABS/blob/main/Module_01_Lab_02_MLPractice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine learning terms and metrics

FMML Module 1, Lab 2<br>


 In this lab, we will show a part of the ML pipeline by extracting features, training and testing

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets
# set randomseed
rng = np.random.default_rng(seed=42)

In this lab, we will use the California Housing dataset. There are 20640 samples, each with 8 attributes like income of the block, age of the houses per district etc. The task is to predict the cost of the houses per district.

Let us download and examine the dataset.

In [ ]:
 dataset =  datasets.fetch_california_housing()
 # print(dataset.DESCR)  # uncomment this if you want to know more about this dataset
 # print(dataset.keys())  # if you want to know what else is there in this dataset
 dataset.target = dataset.target.astype(np.int) # so that we can classify
 print(dataset.data.shape)
 print(dataset.target.shape)

(20640, 8)
(20640,)


<ipython-input-3-60ae2e9a125e>:4: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dataset.target = dataset.target.astype(np.int) # so that we can classify


Here is a function for calculating the 1-nearest neighbours

In [ ]:
def NN1(traindata, trainlabel, query):
  diff  = traindata - query  # find the difference between features. Numpy automatically takes care of the size here
  sq = diff*diff # square the differences
  dist = sq.sum(1) # add up the squares
  label = trainlabel[np.argmin(dist)] # our predicted label is the label of the training data which has the least distance from the query
  return label

def NN(traindata, trainlabel, testdata):
  # we will run nearest neighbour for each sample in the test data
  # and collect the predicted classes in an array using list comprehension
  predlabel = np.array([NN1(traindata, trainlabel, i) for i in testdata])
  return predlabel

We will also define a 'random classifier', which randomly allots labels to each sample

In [ ]:
def RandomClassifier(traindata, trainlabel, testdata):
  # in reality, we don't need these arguments

  classes = np.unique(trainlabel)
  rints = rng.integers(low=0, high=len(classes), size=len(testdata))
  predlabel = classes[rints]
  return predlabel

Let us define a metric 'Accuracy' to see how good our learning algorithm is. Accuracy is the ratio of the number of correctly classified samples to the total number of samples. The higher the accuracy, the better the algorithm.

In [ ]:
def Accuracy(gtlabel, predlabel):
  assert len(gtlabel)==len(predlabel), "Length of the groundtruth labels and predicted labels should be the same"
  correct = (gtlabel==predlabel).sum() # count the number of times the groundtruth label is equal to the predicted label.
  return correct/len(gtlabel)

Let us make a function to split the dataset with the desired probability.

In [ ]:
def split(data, label, percent):
  # generate a random number for each sample
  rnd = rng.random(len(label))
  split1 = rnd<percent
  split2 = rnd>=percent
  split1data = data[split1,:]
  split1label = label[split1]
  split2data = data[split2,:]
  split2label = label[split2]
  return split1data, split1label, split2data, split2label

We will reserve 20% of our dataset as the test set. We will not change this portion throughout our experiments

In [ ]:
testdata, testlabel, alltraindata, alltrainlabel = split(dataset.data, dataset.target, 20/100)
print('Number of test samples = ', len(testlabel))
print('Number of other samples = ', len(alltrainlabel))
print('Percent of test data = ', len(testlabel)*100/len(dataset.target),'%')

Number of test samples =  4144
Number of other samples =  16496
Percent of test data =  20.07751937984496 %


## Experiments with splits

Let us reserve some of our train data as a validation set

In [ ]:
traindata, trainlabel, valdata, vallabel = split(alltraindata, alltrainlabel, 75/100)

What is the accuracy of our classifiers on the train dataset?

In [ ]:
trainpred = NN(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Train accuracy using nearest neighbour is ", trainAccuracy)

trainpred = RandomClassifier(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Train accuracy using random classifier is ", trainAccuracy)

Train accuracy using nearest neighbour is  1.0
Train accuracy using random classifier is  0.164375808538163


For nearest neighbour, the train accuracy is always 1. The accuracy of the random classifier is close to 1/(number of classes) which is 0.1666 in our case.

Let us predict the labels for our validation set and get the accuracy

In [ ]:
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour is ", valAccuracy)

valpred = RandomClassifier(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using random classifier is ", valAccuracy)

Validation accuracy using nearest neighbour is  0.34108527131782945
Validation accuracy using random classifier is  0.1688468992248062


Validation accuracy of nearest neighbour is considerably less than its train accuracy while the validation accuracy of random classifier is the same. However, the validation accuracy of nearest neighbour is twice that of the random classifier.

Now let us try another random split and check the validation accuracy

In [ ]:
traindata, trainlabel, valdata, vallabel = split(alltraindata, alltrainlabel, 75/100)
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy of nearest neighbour is ", valAccuracy)

Validation accuracy of nearest neighbour is  0.34048257372654156


You can run the above cell multiple times to try with different random splits.
We notice that the accuracy is different for each run, but close together.

Now let us compare it with the accuracy we get on the test dataset.

In [ ]:
testpred = NN(alltraindata, alltrainlabel, testdata)
testAccuracy = Accuracy(testlabel, testpred)
print('Test accuracy is ', testAccuracy)

Test accuracy is  0.34917953667953666


### Try it out for yourself and answer:
1. How is the accuracy of the validation set affected if we increase the percentage of validation set? What happens when we reduce it?
2. How does the size of the train and validation set affect how well we can predict the accuracy on the test set using the validation set?
3. What do you think is a good percentage to reserve for the validation set so that thest two factors are balanced?

Answer for both nearest neighbour and random classifier. You can note down the values for your experiments and plot a graph using  <a href=https://matplotlib.org/stable/gallery/lines_bars_and_markers/step_demo.html#sphx-glr-gallery-lines-bars-and-markers-step-demo-py>plt.plot<href>. Check also for extreme values for splits, like 99.9% or 0.1%

#1)How is the accuracy of the validation set affected if we increase the percentage of validation set?



Increasing the Percentage of Validation Set:

Pros:

Better Evaluation: A larger validation set can provide a more reliable estimate of your model's performance. It helps in assessing how well the model generalizes to unseen data.
Detecting Overfitting: With a larger validation set, it becomes easier to detect overfitting. Overfitting occurs when a model learns to perform well on the training data but doesn't generalize well to new data. A larger validation set provides more data for this detection.
Cons:

Reduced Training Data: As you allocate more data to the validation set, you have less data available for training your model. This can potentially lead to underfitting, where the model doesn't learn the underlying patterns in the data well.
Reducing the Percentage of Validation Set:

Pros:

More Data for Training: With a smaller validation set, you have more data available for training your model. This can be beneficial, especially if you have a limited dataset.
Cons:

Less Reliable Evaluation: A smaller validation set may provide a less reliable estimate of your model's performance. The evaluation might be more sensitive to random variations in the validation data, leading to less confidence in the results.
Risk of Overfitting: With less data for validation, it becomes harder to detect overfitting. The model might appear to perform well on the validation set, but it may not generalize well to new data.
The choice of the percentage of data allocated to the validation set is a trade-off. It depends on factors like the size of your dataset, the complexity of your model, and your specific goals. Common practices often involve splitting the data into 70-80% for training and 20-30% for validation in many machine learning projects. However, there are no strict rules, and you may need to experiment with different splits to find the one that works best for your particular problem.




#2)How does the size of the train and validation set affect how well we can predict the accuracy on the test set using the validation set?

Small Training Set, Large Validation Set:

High Bias, Low Variance: When you have a small training set and a large validation set, your model may underfit the data. It doesn't have enough information to learn the underlying patterns, so it will perform poorly on both the training and validation sets. In this case, the validation set won't be a good indicator of test set performance.
Large Training Set, Small Validation Set:

Low Bias, High Variance: With a large training set and a small validation set, your model might overfit the training data. It can memorize the training examples, but this memorization doesn't generalize well to unseen data. In this case, the validation set might also not be a reliable indicator of test set performance.
Balanced Training and Validation Sets:

Optimal Trade-off: Ideally, you want a balanced setup where both the training and validation sets are reasonably sized. This helps strike a balance between bias and variance. The model learns enough from the training data to capture the underlying patterns but doesn't overfit. The validation set provides a good estimate of how well the model generalizes to unseen data, making it a better predictor of test set accuracy.
Cross-Validation:

Robust Evaluation: To mitigate the impact of the size of the validation set, you can also use techniques like k-fold cross-validation. This involves splitting the data into multiple subsets (folds) and training and validating the model multiple times, rotating through different subsets as validation data. This provides a more robust estimate of model performance and reduces the variability associated with the size of a single validation set.

# 3)What do you think is a good percentage to reserve for the validation set so that thest two factors are balanced?

80/20 or 70/30 Split (Train/Validation): In many cases, an 80/20 or 70/30 split is a good starting point. This means you allocate 80% (or 70%) of your data for training and the remaining 20% (or 30%) for validation. This is a common split when you have a moderate-sized dataset.

Cross-Validation: If you have a limited amount of data, you might want to consider k-fold cross-validation, where you divide your data into k subsets (folds) and perform training and validation k times, rotating through the folds. This allows you to use all of your data for both training and validation and provides a more robust estimate of model performance.

Stratified Split: If your dataset has class imbalance (e.g., one class has significantly fewer samples than others), consider using stratified sampling to ensure that both the training and validation sets maintain the same class distribution as the original dataset. This can be crucial for accurate evaluation.

Data Size Considerations: If you have a very large dataset, you can allocate a smaller percentage to the validation set, such as 10% or even less, and still have enough data for training. Conversely, if your dataset is very small, you may need to allocate a larger percentage to validation to ensure you have a representative sample for evaluation.

Hyperparameter Tuning: If you plan to perform extensive hyperparameter tuning, you might consider setting aside a larger validation set, as this will require more data for robust validation.

## Multiple Splits

One way to get more accurate estimates for the test accuracy is by using <b>crossvalidation</b>. Here, we will try a simple version, where we do multiple train/val splits and take the average of validation accuracies as the test accuracy estimation. Here is a function for doing this. Note that this function will take a long time to execute.

In [ ]:
# you can use this function for random classifier also
def AverageAccuracy(alldata, alllabel, splitpercent, iterations, classifier=NN):
  accuracy = 0
  for ii in range(iterations):
    traindata, trainlabel, valdata, vallabel = split(alldata, alllabel, splitpercent)
    valpred = classifier(traindata, trainlabel, valdata)
    accuracy += Accuracy(vallabel, valpred)
  return accuracy/iterations # average of all accuracies

In [ ]:
print('Average validation accuracy is ', AverageAccuracy(alltraindata, alltrainlabel, 75/100, 10, classifier=NN))
testpred = NN(alltraindata, alltrainlabel, testdata)
print('test accuracy is ',Accuracy(testlabel, testpred) )

Average validation accuracy is  0.33584635395170215
test accuracy is  0.34917953667953666


This is a very simple way of doing cross-validation. There are many well-known algorithms for cross-validation, like k-fold cross-validation, leave-one-out etc. This will be covered in detail in a later module. For more information about cross-validation, check <a href=https://en.wikipedia.org/wiki/Cross-validation_(statistics)>Cross-validatioin (Wikipedia)</a>

### Questions
1. Does averaging the validation accuracy across multiple splits give more consistent results?
2. Does it give more accurate estimate of test accuracy?
3. What is the effect of the number of iterations on the estimate? Do we get a better estimate with higher iterations?
4. Consider the results you got for the previous questions. Can we deal with a very small train dataset or validation dataset by increasing the iterations?


# 1)Does averaging the validation accuracy across multiple splits give more consistent results?

Yes, averaging the validation accuracy across multiple splits of your data can often give more consistent and reliable results compared to relying on a single split. This practice is known as cross-validation, and it helps assess the performance and generalization of a machine learning model more robustly.

# 2)Does it give more accurate estimate of test accuracy?

Cross-validation, while it provides a more robust estimate of model performance compared to a single train-validation split, is still primarily an estimate of how your model might perform on unseen data. It is not a direct estimate of the model's performance on a true, independent test dataset. The purpose of cross-validation is to assess the model's generalization ability and help with hyperparameter tuning, but it doesn't replace the need for a final evaluation on a dedicated test set.

# 3)What is the effect of the number of iterations on the estimate? Do we get a better estimate with higher iterations?

The number of iterations, or folds, in a cross-validation procedure can have an impact on the estimate of model performance. Generally, using a higher number of iterations can lead to a more robust estimate, but there are trade-offs to consider:

More Stable Estimate: Increasing the number of iterations in cross-validation (e.g., using a higher value of k in k-fold cross-validation) provides a more stable and reliable estimate of model performance. This is because you're repeatedly training and evaluating the model on different subsets of your data, which helps reduce the impact of random variations in the data.

Smaller Training Sets: However, as you increase the number of iterations, each training fold becomes smaller because the total number of data points is divided into more subsets. Smaller training sets may lead to models that are less representative of the overall data distribution.

Computational Cost: Running cross-validation with a higher number of iterations can be computationally expensive, especially if your dataset is large or if the model training process is time-consuming. You need to balance the computational cost with the desire for a more accurate estimate.

Bias-Variance Trade-off: The choice of the number of iterations can be seen as a bias-variance trade-off. With a larger number of iterations, you reduce the variance in your estimate (resulting in a more stable estimate), but you might introduce a small bias because each training set is smaller.

# 4)Consider the results you got for the previous questions. Can we deal with a very small train dataset or validation dataset by increasing the iterations?

Increasing the number of iterations in a cross-validation procedure can help mitigate some of the issues associated with very small training or validation datasets to a certain extent, but it may not fully address the underlying problems. Here's how increasing iterations can help and what limitations still apply:

Advantages of Increasing Iterations:

Improved Stability: With more iterations, you obtain multiple estimates of model performance, which can help reduce the impact of randomness in the data. This improved stability can provide a more reliable assessment of how your model generalizes.

Better Utilization of Data: If you have a very small dataset, using more iterations (e.g., increasing k in k-fold cross-validation) can allow you to use a larger portion of your data for training and validation in total. This can help your model learn from more data, which is beneficial.

Limitations and Considerations:

Small Training Sets: Increasing the number of iterations does not change the fact that each individual training set within a fold becomes smaller as you divide your data into more subsets. Small training sets can lead to models that may not generalize well because they haven't seen enough diverse examples.

Validation Set Size: The size of the validation set remains constant regardless of the number of iterations. If you have a very small validation set, it can make it challenging to obtain reliable estimates of model performance because the evaluation might be sensitive to the specific data points in the validation set.

Computational Cost: Running cross-validation with a large number of iterations can become computationally expensive, especially if the model training process is time-consuming.

Bias-Variance Trade-off: Increasing iterations may reduce variance in the estimate but could introduce a small bias due to smaller training sets within each fold.

In cases where you have a very small dataset, it's important to be cautious and consider other strategies as well:

Data Augmentation: If possible, augment your dataset by creating variations of existing data points to artificially increase the size of your training dataset.

Use Simpler Models: With a limited amount of data, it may be beneficial to use simpler models (with fewer parameters) to avoid overfitting.

Regularization: Apply regularization techniques to prevent overfitting in cases of small datasets.

Collect More Data: If feasible, consider collecting additional data to increase the size of your dataset, which can lead to more robust model training and evaluation.